# Lab 13d: Let's build an Observable Markov Model of a Stock Returns
This lab will familiarize students with constructing an Observable Markov Model of the excess return of a ticker `XYZ` where we defined the excess return as:

$$
\begin{equation*}
R_{ij} = \left(\frac{1}{\Delta{t}}\right)\cdot\ln\left(\frac{S_{i,j}}{S_{i,j-1}}\right) - \bar{r}_{f}
\end{equation*}
$$

where $R_{ij}$ denotes the excess return of equity $i$ at time $j$, $\Delta{t}$ denotes the time between $j-1\rightarrow{j}$ (units: years), $S_{i,\star}$ denotes the share price of equity $i$ at time $\star$, and $\bar{r}_{f}$ denotes the annualized risk free rate.

## Tasks
Describe the day-to-day variation of the excess return using a fully observable Markov model described by the tuple $\mathcal{M} = (\mathcal{S},\mathcal{O},\mathbf{P},\mathbf{E})$; $\mathcal{S}$ is the set of hidden states, $\mathcal{O}$ is the set of observable states, $\mathbf{P}$ is the transition matrix and $\mathbf{E}$ is the emission matrix. Because we are fully observable, the emission matrix $\mathbf{E} = \mathbf{I}$, where $\mathbf{I}$ is the identity matrix.

* `Task 1`: Construct the states $\mathcal{S}$, the emission matrix $\mathbf{E}$ and transition matrix $\hat{\mathbf{T}}$
    * `TODO`: Estimate the transition matrix $\hat{\mathbf{T}}$ from market data
* `Task 2`: Simulate the Observable Markov Model (OMM) for an average trading year
    * `TODO`: Generate the stationary distribution from the estimated $\hat{\mathbf{T}}$ matrix

## Setup

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Labs-AY-2024/week-13/Lab-13d`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Labs-AY-2024/week-13/Lab-13d/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Labs-AY-2024/week-13/Lab-13d/Manifest.toml`
Precompiling project...
  ✓ DataStructures
  ✓ SortingAlgorithms
  ✓ QuadGK
  ✓ StatsBase
  ✓ Distributions
  ✓ DataFrames
  6 dependencies successfully precompiled in 18 seconds. 55 already precompiled.
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Labs-AY-2024/week-13/Lab-13d/Project.toml`
    Updating `~/Desktop/julia_work/CHEME-4800-5800-Labs-AY-2024/week-13/Lab-13d/Manifest.toml`
  [864edb3b] ↑ DataStructures v0.18.19 ⇒ v0.18.20
Precompiling project...
  ✓ DataFrames
  1 dependency successfully precompiled in 15 seconds. 60 already precompiled.
        Info We haven't cleaned this depot up for a bit, running Pkg.gc()...
      Active

## Prerequisites 

In [2]:
# initialize -
dataset = Dict{String,DataFrame}();
risk_free_rate = 0.05;

# load the price dataset full dataset, remove firms with missing data -
original_dataset = MyPortfolioDataSet() |> x->x["dataset"];
maximum_number_trading_days = original_dataset["AAPL"] |> nrow;
for (ticker,data) ∈ original_dataset
    if (nrow(data) == maximum_number_trading_days)
        dataset[ticker] = data;
    end
end
my_list_of_tickers = keys(dataset) |> collect |> x->sort(x);

# what ticker?
ticker = "SPY"
idx_ticker = findfirst(x->x==ticker, my_list_of_tickers);

# compute the growth rate matrix -
market_matrix = μ(dataset, my_list_of_tickers, risk_free_rate = risk_free_rate) |> x-> transpose(x) |> Matrix;
Rₘ = market_matrix[idx_ticker, :]; # this is growth rate of the price

## Task 1: Construct the states $\mathcal{S}$, the emission matrix $\mathbf{E}$ and transition matrix $\hat{\mathbf{T}}$
First, consider the states $\mathcal{S}$. Suppose we put a label on each excess return value, ranging from `super bad,` $\dots$,` unchanged,` $\dots$,` super good,` where if $R\ll{0}$, then we are `super bad,` or $R\gg{0}$ we are in the `super good` category (or we are someplace in between). 
* We could use the [cumulative distribution function](https://en.wikipedia.org/wiki/Cumulative_distribution_function) computed from the observed return series $R_{i,1}, \dots, R_{i,n}$ to partition actual excess returns into one of a fixed number of categories. Once we have the categories, we can compute the probability that category $i$ is followed by category $j$ (these values are entries in the state transition matrix $\hat{\mathbf{T}}$).
* To start, specify a value for the  `number_of_states` variable, where the `number_of_states` controls how many categories we have when splitting up the return time series

In [3]:
number_of_states = 5;
states = range(1,stop=number_of_states) |> collect;

Next, set up the emissions matrix $\mathbf{E}$. For this example, we assume that the states are __fully observable__, i.e., we can see the states directly. Thus, the emission matrix $\mathbf{E}$ is the identity matrix $\mathbf{I}$:

In [4]:
E = diagm(ones(number_of_states));

### TODO: Estimate the transition matrix $\hat{\mathbf{T}}$ from market data
To estimate the transition matrix $\hat{\mathbf{T}}$, we'll estimate the transition probabilities from the excess return data calculated in the `Prerequisites` section and saved in the `Rₘ` variable:
* Let's split the data into two blocks: the first (which we'll call `in-sample`) will be used to estimate the elements of the matrix $\hat{\mathbf{T}}$, while the second (which we'll call `out-of-sample`) will be used for testing purposes.

In [5]:
split_fraction = 0.90;
insample_end_index = round(split_fraction*length(Rₘ),digits=0) |> Int
in_sample_dataset = Rₘ[1:insample_end_index]
out_of_sample_dataset = Rₘ[(insample_end_index+1):end];

The excess return distribution is a random variable that follows some [probability distribution function](https://en.wikipedia.org/wiki/Probability_distribution). We don't know what that distribution is, but for now, we assume the excess returns follow a [Laplace distribution](https://en.wikipedia.org/wiki/Laplace_distribution)
* We use the [fit_mle function](https://juliastats.org/Distributions.jl/stable/fit/#Distributions.fit_mle-Tuple{Any,%20Any}) exported by the [Distributions.jl package](https://github.com/JuliaStats/Distributions.jl) to fit a [Laplace distribution](https://en.wikipedia.org/wiki/Laplace_distribution) to our our data. We fit the distribution to the full dataset `Rₘ`, and save the distribution in the `d` variable.

In [6]:
d = fit_mle(Laplace, Rₘ); # use the *full* data set to establish the cutoff's

Next, we generate the percentile cutoffs that we use to establish the bounds that correspond to each category of return, i.e., `super bad` or `super good`, etc.

In [7]:
percentage_cutoff = range(0.0,stop=1.0,length=(number_of_states+1)) |> collect;

Now that we have the cutoffs, compute the lower and upper bound for each potentiual category. To do this, we'll use the [quantile function](https://juliastats.org/Distributions.jl/stable/univariate/#Statistics.quantile-Tuple{UnivariateDistribution,%20Real}) exported by the [Distributions.jl package](https://github.com/JuliaStats/Distributions.jl). For a given `0 ≤ q ≤ 1`, `quantile(d, q)` is the smallest value `x` 
for which `cdf(d, x) ≥ q`.

In [15]:
bounds = Array{Float64,2}(undef, number_of_states,2)
for s ∈ states
    bounds[s,1] = quantile(d,percentage_cutoff[s])
    bounds[s,2] = quantile(d,percentage_cutoff[s+1])
end
bounds;

Now that we have the category bounds, let's take the excess return data and determine which state an excess return observation corresponds to. For each sample in the `in_sample_dataset`:
* Classify the sample value into one of the possible categories. Let `state = 1` equal the worst return, and `state = number_of_states` equal the best return. Save these results in the `encoded_in_sample` array:

In [9]:
encoded_in_sample = Array{Int64,1}();
for i ∈ eachindex(in_sample_dataset)
    value = in_sample_dataset[i];

    class_index = 1;
    for s ∈ states
        if (bounds[s,1] ≤ value && value < bounds[s,2])
            class_index = s;
            break;
        end
    end
    push!(encoded_in_sample, class_index);
end
encoded_in_sample;

In the matrix $\mathbf{T}$ compute the `counts` for transition from state `i` to state `j`:

In [10]:
T = zeros(number_of_states, number_of_states)
for i ∈ 2:insample_end_index
    start_index = encoded_in_sample[i-1];
    stop_index = encoded_in_sample[i];
    T[start_index,stop_index] += 1;
end
T

5×5 Matrix{Float64}:
 110.0  44.0  28.0  44.0  75.0
  61.0  57.0  46.0  63.0  29.0
  46.0  55.0  77.0  57.0  36.0
  39.0  60.0  74.0  68.0  58.0
  45.0  40.0  47.0  66.0  75.0

From the `counts` matrix $\mathbf{T}$, compute the transtion probability matrix $\hat{\mathbf{T}}$:

In [11]:
T̂ = zeros(number_of_states, number_of_states)
for row ∈ states
    Z = sum(T[row,:]);
    for col ∈ states
        T̂[row,col] = (1/Z)*T[row,col]
    end
end
T̂

5×5 Matrix{Float64}:
 0.365449  0.146179  0.0930233  0.146179  0.249169
 0.238281  0.222656  0.179688   0.246094  0.113281
 0.169742  0.202952  0.284133   0.210332  0.132841
 0.130435  0.200669  0.247492   0.227425  0.19398
 0.164835  0.14652   0.172161   0.241758  0.274725

## Task 2: Simulate the Observable Markov Model (OMM)

In [16]:
model = build(MyObservableMarkovModel, (
    states = states,
    T = T̂, 
    E = E
));

### TODO: Generate the stationary distribution from the estimated $\hat{\mathbf{T}}$ matrix
Generate the stationary distribution for the estimated transition matrix $\hat{T}$ and use it to construct a [Categorical distribution](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Categorical) representing the stationary distrubution, save the [Categorical distribution](https://juliastats.org/Distributions.jl/stable/univariate/#Distributions.Categorical) in the `π̄`-variable:

In [13]:
π̄ = (T̂^100) |> tmp -> Categorical(tmp[1,:]);

In [14]:
number_of_paths = 10;
number_of_steps = 252; # average number of trading days per year
archive = Array{Int64,2}(undef, number_of_steps, number_of_paths);
for i ∈ 1:number_of_paths
    start_state = rand(π̄);
    tmp = model(start_state, number_of_steps)
    for j ∈ 1:number_of_steps
        archive[j,i] = tmp[j]
    end
end
archive

252×10 Matrix{Int64}:
 3  2  3  5  1  3  3  3  5  1
 1  5  5  1  1  3  4  2  1  2
 4  3  5  5  1  4  4  5  1  2
 1  3  4  5  1  5  5  5  5  4
 1  1  3  5  1  5  1  1  3  3
 5  1  3  5  5  1  5  1  4  5
 2  1  3  2  5  1  4  5  4  2
 4  4  1  2  1  5  3  5  1  2
 4  3  1  4  2  5  4  2  4  2
 1  3  1  3  3  5  2  4  4  1
 1  3  2  1  2  5  4  3  4  3
 4  1  3  2  4  4  2  2  5  5
 3  1  1  4  4  1  3  3  3  2
 ⋮              ⋮           
 4  4  2  5  1  4  5  2  3  4
 4  5  5  5  4  3  5  3  1  4
 2  3  5  3  4  2  3  5  1  5
 2  2  4  2  1  2  3  4  1  5
 3  3  2  1  4  1  1  3  4  1
 5  2  5  4  5  1  1  5  4  2
 1  1  2  2  4  3  1  3  2  5
 3  1  5  5  4  4  2  5  1  5
 3  2  4  5  4  3  4  3  1  5
 4  3  4  2  4  3  3  5  1  2
 5  2  5  2  5  5  3  4  3  4
 4  3  5  3  4  3  3  5  5  2